In [11]:
# 1. Setup
import pandas as pd
import numpy as np
import pickle
import mlflow
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline


In [12]:
# 2. Load Train/Test Data

train_df = pd.read_csv("../data/processed/train.csv")
test_df = pd.read_csv("../data/processed/test.csv")

# Target
target = "churn"
X_train = train_df.drop(columns=[target])
y_train = train_df[target]

X_test = test_df.drop(columns=[target])
y_test = test_df[target]

In [13]:
print(f"✅ Train shape: {train_df.shape}")
print(f"✅ Test shape: {test_df.shape}")

# Optional: show a few rows
train_df.head()

✅ Train shape: (7088, 19)
✅ Test shape: (3039, 19)


,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Total_Trans_Amt,Total_Trans_Ct,Total_Amt_Chng_Q4_Q1,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Gender,Education_Level,Marital_Status,Income_Category,Card_Category,churn
0,44,3,36,2,3,3,6680.0,1839,7632,95,0.617,0.532,0.275,F,Uneducated,Married,Less than $40K,Blue,0
1,39,1,34,3,1,1,2884.0,2517,4809,87,0.693,0.740,0.873,F,Graduate,Single,Unknown,Blue,0
2,52,1,36,4,2,2,14858.0,1594,4286,72,0.510,0.636,0.107,M,Unknown,Married,$80K - $120K,Blue,0
3,34,0,17,4,1,4,2638.0,2092,1868,43,0.591,0.344,0.793,M,Graduate,Married,$40K - $60K,Blue,0
4,47,5,36,3,1,2,8896.0,1338,4252,70,0.741,0.591,0.150,M,Doctorate,Single,Less than $40K,Blue,0


In [10]:
# Set MLflow tracking URI and experiment
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("churn-pipeline-baseline-models")

2025/07/10 10:08:32 INFO mlflow.tracking.fluent: Experiment with name 'churn-pipeline-baseline-models' does not exist. Creating a new experiment.


<Experiment: artifact_location='/workspaces/churn-clv-mlops-project/mlruns/1', creation_time=1752142112563, experiment_id='1', last_update_time=1752142112563, lifecycle_stage='active', name='churn-pipeline-baseline-models', tags={}>

In [14]:


# === 1. Define your custom feature engineering logic ===
def add_interaction_features(df):
    df = df.copy()
    df["Avg_Transaction_Amt"] = df["Total_Trans_Amt"] / (df["Total_Trans_Ct"] + 1e-3)
    df["Revolve_to_Limit"] = df["Total_Revolving_Bal"] / (df["Credit_Limit"] + 1e-3)
    df["AmtCt_Chg_Ratio"] = df["Total_Amt_Chng_Q4_Q1"] / (df["Total_Ct_Chng_Q4_Q1"] + 1e-3)
    return df

# Wrap it as a FunctionTransformer
interaction_transformer = FunctionTransformer(add_interaction_features)

# === 2. Separate features ===
categorical = [
    "Gender", "Education_Level", "Marital_Status", 
    "Income_Category", "Card_Category"
]

numerical_to_scale = [
    "Credit_Limit", "Total_Revolving_Bal", 
    "Total_Trans_Amt", "Total_Trans_Ct", 
    "Avg_Utilization_Ratio", "Avg_Transaction_Amt", 
    "Revolve_to_Limit", "AmtCt_Chg_Ratio"
]

numerical_no_scale = [
    "Customer_Age", "Dependent_count", "Months_on_book",
    "Total_Relationship_Count", "Months_Inactive_12_mon",
    "Contacts_Count_12_mon", "Total_Amt_Chng_Q4_Q1", 
    "Total_Ct_Chng_Q4_Q1"
]

# === 3. Define transformers ===

cat_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

num_scale_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

num_noscale_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median"))
])

# === 4. Compose all into one ColumnTransformer ===
preprocessor = ColumnTransformer([
    ("num_scaled", num_scale_transformer, numerical_to_scale),
    ("num_noscale", num_noscale_transformer, numerical_no_scale),
    ("cat", cat_transformer, categorical)
])

# === 5. Final pipeline with feature engineering + preprocessor ===
full_pipeline = Pipeline([
    ("feature_engineering", interaction_transformer),
    ("preprocessor", preprocessor)
])

In [15]:
def train_and_log_model(model, model_name, extra_params=None):
    #mlflow.set_tracking_uri("http://127.0.0.1:5000")
    #mlflow.set_experiment("churn-pipeline-baseline-models")
    
    with mlflow.start_run():
        mlflow.set_tag("model", model_name)
        
        # Automatically extract model parameters
        model_params = model.get_params()
        
        # Merge with any manually passed params (optional)
        if extra_params:
            model_params.update(extra_params)
        
        mlflow.log_params(model_params)

        pipeline = Pipeline([
            ("feature_pipeline", full_pipeline),
            ("classifier", model)
        ])

        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        y_proba = pipeline.predict_proba(X_test)[:, 1]

        acc = accuracy_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_proba)

        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("roc_auc", roc_auc)

        mlflow.sklearn.log_model(pipeline, artifact_path="model")

        print(f"✅ {model_name} - Accuracy: {acc:.4f}, ROC AUC: {roc_auc:.4f}")

In [16]:
xgb_model = XGBClassifier(n_estimators=100, max_depth=4, learning_rate=0.1, use_label_encoder=False, eval_metric="logloss")

train_and_log_model(xgb_model, model_name="XGBoost")

/opt/conda/envs/churn-clv-mlops/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [10:30:23] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
2025/07/10 10:30:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/10 10:30:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ XGBoost - Accuracy: 0.9664, ROC AUC: 0.9919
🏃 View run loud-skink-247 at: http://127.0.0.1:5000/#/experiments/1/runs/795c25dd71e644f7aa13bebd21c38a02
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
